# 介绍

# 主要组件
- 线程对象
- Lock对象
- RLock对象
- 信号对象
- 条件对象
- 事件对象

In [ ]:
import threading
from threading import *
import time
import logging
import random

# 定义一个线程
1. 自定义函数后，传给Thread实例
2. 继承`Thread`类后，重写`run`以及其他类方法

In [64]:
def function(i):
    print(f'functoin called by thread {i}')
    return 


for i in range(10):
    t = Thread(target=function, args=(i,))
    t.start()
#     t.join()

functoin called by thread 0
functoin called by thread 1
functoin called by thread 2
functoin called by thread 3
functoin called by thread 4
functoin called by thread 5
functoin called by thread 6
functoin called by thread 7
functoin called by thread 8
functoin called by thread 9


# 确定当前线程

In [72]:
def first_function():
    print(current_thread().name + str(' is starting '))
    time.sleep(1.5)
    print(current_thread().name + str(' is exiting '))
    
def second_function():
    print(current_thread().name + str(' is starting '))
    time.sleep(1.3)
    print(current_thread().name + str(' is exiting '))
    
def third_function():
    print(current_thread().name + str(' is starting '))
    time.sleep(1.2)
    print(current_thread().name + str(' is exiting '))

In [73]:
t1 = Thread(target=first_function, name='first_function')
t2 = Thread(target=second_function, name='second_function')
t3 = Thread(target=third_function, name='third_function')
t1.start()
t2.start()
t3.start()

first_function is starting 
second_function is starting 
third_function is starting 
third_function is exiting 
second_function is exiting 
first_function is exiting 


# 自定义一个线程
- 继承`Thread`类
- 重写初始化类方法
- 重写`run`方法实现自定义

In [2]:
exit_flag = 0

class MyThread(Thread):
    def __init__(self, threadID, name, counter):
        Thread.__init__(self)
#         super().__init__()  # work
#         super().__init__(self)  # wrong
        self.threadID = threadID
        self.name = name
        self.counter = counter
        
    def run(self):
        print('Starting ' + self.name)
        print_time(self.name, self.counter, 5)
        print(f'Exiting {self.name}')
        
def print_time(thread_name, delay, counter):
    while counter:
        if exit_flag:
            import _thread
            _thread.exit()
        time.sleep(delay)
        print(f'{thread_name}: {time.ctime(time.time())}')
        counter -= 1

In [3]:
thread1 = MyThread(1, 'thread-1', 1)
thread2 = MyThread(2, 'thread-2', 2)

thread1.start()
thread2.start()
print(f'exiting {current_thread().name}')

Starting thread-1
Starting thread-2
exiting MainThread
thread-1: Tue Dec 10 11:06:36 2019
thread-1: Tue Dec 10 11:06:37 2019
thread-2: Tue Dec 10 11:06:37 2019
thread-1: Tue Dec 10 11:06:38 2019
thread-1: Tue Dec 10 11:06:39 2019
thread-2: Tue Dec 10 11:06:39 2019
thread-1: Tue Dec 10 11:06:40 2019
Exiting thread-1
thread-2: Tue Dec 10 11:06:41 2019
thread-2: Tue Dec 10 11:06:43 2019
thread-2: Tue Dec 10 11:06:45 2019
Exiting thread-2


In [4]:
thread1 = MyThread(1, 'thread-1', 1)
thread2 = MyThread(2, 'thread-2', 2)

thread1.start()
thread2.start()
thread1.join()
thread2.join()
print(f'exiting {current_thread().name}')

Starting thread-1
Starting thread-2
thread-1: Tue Dec 10 11:06:52 2019
thread-1: Tue Dec 10 11:06:53 2019
thread-2: Tue Dec 10 11:06:53 2019
thread-1: Tue Dec 10 11:06:54 2019
thread-1: Tue Dec 10 11:06:55 2019
thread-2: Tue Dec 10 11:06:55 2019
thread-1: Tue Dec 10 11:06:56 2019
Exiting thread-1
thread-2: Tue Dec 10 11:06:57 2019
thread-2: Tue Dec 10 11:06:59 2019
thread-2: Tue Dec 10 11:07:01 2019
Exiting thread-2
exiting MainThread


# 线程同步

## 使用`Lock`进行线程同步
- ![有死锁出现的情况](https://python-parallel-programmning-cookbook.readthedocs.io/zh_CN/latest/_images/deadlock.png)死锁
- `lock()`：将共享资源某一个时刻的访问限制在单一线程或单一类型的线程上

In [4]:
shared_resource_with_lock = 0
shared_resource_without_lock = 0
COUNT = 100000
shared_resource_lock = Lock()

In [5]:
def increment_with_lock():
    global shared_resource_with_lock
    for _ in range(COUNT):
        shared_resource_lock.acquire()
        shared_resource_with_lock += 1
        shared_resource_lock.release()
        
def decrement_with_lock():
    global shared_resource_with_lock
    for _ in range(COUNT):
        shared_resource_lock.acquire()
        shared_resource_with_lock -= 1
        shared_resource_lock.release()
        
def increment_without_lock():
    global shared_resource_without_lock
    for _ in range(COUNT):
        shared_resource_without_lock += 1
        
def decrement_without_lock():
    global shared_resource_without_lock
    for _ in range(COUNT):
        shared_resource_without_lock -= 1

In [8]:
t1 = Thread(target=increment_with_lock)
t2 = Thread(target=decrement_with_lock)
t3 = Thread(target=increment_without_lock)
t4 = Thread(target=decrement_without_lock)
t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print(f"Exit {current_thread().name}")
print(f'shared_resource_with_lock value is: {shared_resource_with_lock}')
print(f'shared_resource_without_lock value is: {shared_resource_without_lock}')

Exit MainThread
shared_resource_with_lock value is: 0
shared_resource_without_lock value is: -51913


In [73]:
help(type(Lock()))

Help on class lock in module _thread:

class lock(builtins.object)
 |  A lock object is a synchronization primitive.  To create a lock,
 |  call threading.Lock().  Methods are:
 |  
 |  acquire() -- lock the lock, possibly blocking until it can be obtained
 |  release() -- unlock of the lock
 |  locked() -- test whether the lock is currently locked
 |  
 |  A lock is not owned by the thread that locked it; another thread may
 |  unlock it.  A thread attempting to lock a lock that it has already locked
 |  will block until another thread unlocks it.  Deadlocks may ensue.
 |  
 |  Methods defined here:
 |  
 |  __enter__(...)
 |      acquire(blocking=True, timeout=-1) -> bool
 |      (acquire_lock() is an obsolete synonym)
 |      
 |      Lock the lock.  Without argument, this blocks if the lock is already
 |      locked (even by the same thread), waiting for another thread to release
 |      the lock, and return True once the lock is acquired.
 |      With an argument, this will only b

In [72]:
shared_resource_lock.locked()

False

## 使用`RLock`进行线程同步
- 只有个拿到锁的线程才能释放该锁
- 和`Lock`的区别
    - 谁拿到谁释放。如果线程A拿到锁，线程B无法释放这个锁，只有A可以释放
    - 同一线程可以多次拿到该锁，即可以acquire多次
    - acquire多少次就必须release多少次，只有最后一次release才能改变RLock的状态为unlocked）

In [9]:
class Box:
    lock = RLock()
    
    def __init__(self):
        self.total_items = 0
        
    def execute(self, n):
        self.lock.acquire()
        self.total_items += n
        self.lock.release()
        
    def add(self):
        self.lock.acquire()
        self.execute(1)
        self.lock.release()
        
    def remove(self):
        self.lock.acquire()
        self.execute(-1)
        self.lock.release()
        
def adder(box, items):
    while items > 0:
        print('adding 1 item in the box')
        box.add()
        time.sleep(1)
        items -= 1
        
def remover(box, items):
    while items > 0:
        print('removing 1 item from the box')
        box.remove()
        time.sleep(1)
        items -= 1

In [10]:
items = 5
print(f'Putting {items} items in the box')
box = Box()
t1 = Thread(target=adder, args=(box, items))
t2 = Thread(target=remover, args=(box, items))
t1.start()
t2.start()
t1.join()
t2.join()
print(f'{box.total_items} items still in the box')
print(f"{current_thread().name} exited")

Putting 5 items in the box
adding 1 item in the box
removing 1 item from the box
adding 1 item in the box
removing 1 item from the box
adding 1 item in the box
removing 1 item from the box
adding 1 item in the box
removing 1 item from the box
adding 1 item in the box
removing 1 item from the box
0 items still in the box
MainThread exited


## 使用信号量进行线程同步
- 每当线程想要读取关联了信号量的共享资源时，必须调用 acquire() ，此操作减少信号量的内部变量, 如果此变量的值非负，那么分配该资源的权限。如果是负值，那么线程被挂起，直到有其他的线程释放资源。
- 当线程不再需要该共享资源，必须通过 release() 释放。这样，信号量的内部变量增加，在信号量等待队列中排在最前面的线程会拿到共享资源的权限。
- ![pics](https://python-parallel-programmning-cookbook.readthedocs.io/zh_CN/latest/_images/semaphores.png)
- 缺陷：非原子操作有可能会导致所有线程处于等待状态

In [3]:
semaphore = Semaphore(0)

def consumer():
    print('consumer is waiting...')
    semaphore.acquire()
    print(f'Consumer notify: cosumed item number {item}')
    
def producer():
    global item
    time.sleep(3)
    item = random.randint(0,1000)
    print(f'Producer notify: produced item number {item}')
    semaphore.release()

In [4]:
for _ in range(5):
    t1 = Thread(target=producer)
    t2 = Thread(target=consumer)
    t1.start()
    t2.start()
    t1.join()
    t2.join()
print(f"{current_thread().name} exited")

consumer is waiting...
Producer notify: produced item number 472
Consumer notify: cosumed item number 472
consumer is waiting...
Producer notify: produced item number 408
Consumer notify: cosumed item number 408
consumer is waiting...
Producer notify: produced item number 14
Consumer notify: cosumed item number 14
consumer is waiting...
Producer notify: produced item number 444
Consumer notify: cosumed item number 444
consumer is waiting...
Producer notify: produced item number 972
Consumer notify: cosumed item number 972
MainThread exited


## 使用条件进行线程同步
- 条件指的是应用程序状态的改变。这是另一种同步机制，其中某些线程在等待某一条件发生，其他的线程会在该条件发生的时候进行通知。一旦条件发生，线程会拿到共享资源的唯一权限
- 解释条件机制最好的例子还是生产者-消费者问题。在本例中，只要缓存不满，生产者一直向缓存生产；只要缓存不空，消费者一直从缓存取出（之后销毁）。当缓冲队列不为空的时候，生产者将通知消费者；当缓冲队列不满的时候，消费者将通知生产者

In [6]:
items = []
condition = Condition()

In [14]:
class Consumer(Thread):
    def __init__(self):
        super().__init__()
        
    def consume(self):
        global condition
        global items
        condition.acquire()
        if not len(items):
            condition.wait()
            print('Consumer notify: no item to consume')
        items.pop()
        print('Consumer notify: consumed 1 item')
        print(f'Comsumer notify: items left to consume are {str(len(items))}')
        
        condition.notify()
        condition.release()
        
    def run(self):
        for _ in range(10):
            time.sleep(1)
            self.consume()
            
class Producer(Thread):
    def __init__(self):
        super().__init__()
        
    def produce(self):
        global condition
        global items
        condition.acquire()
        if len(items) == 5:
            print('full.....')
            condition.wait()
            print(f'Producer notify: items producted are {str(len(items))}')
            print('Producer notify: stop the production!')
        items.append(1)
        print(f'Producer notify: total items producted {str(len(items))}')
        condition.notify()
        condition.release()
        
    def run(self):
        for _ in range(10):
            time.sleep(.5)
            self.produce()

In [15]:
producer = Producer()
consumer = Consumer()
producer.start()
consumer.start()
producer.join()
consumer.join()
print(f"{current_thread().name} exited...")

Producer notify: total items producted 1
Consumer notify: consumed 1 item
Comsumer notify: items left to consume are 0
Producer notify: total items producted 1
Producer notify: total items producted 2
Consumer notify: consumed 1 item
Comsumer notify: items left to consume are 1
Producer notify: total items producted 2
Producer notify: total items producted 3
Consumer notify: consumed 1 item
Comsumer notify: items left to consume are 2
Producer notify: total items producted 3
Producer notify: total items producted 4
Consumer notify: consumed 1 item
Comsumer notify: items left to consume are 3
Producer notify: total items producted 4
Producer notify: total items producted 5
Consumer notify: consumed 1 item
Comsumer notify: items left to consume are 4
Producer notify: total items producted 5
Consumer notify: consumed 1 item
Comsumer notify: items left to consume are 4
Consumer notify: consumed 1 item
Comsumer notify: items left to consume are 3
Consumer notify: consumed 1 item
Comsumer no

![process](https://python-parallel-programmning-cookbook.readthedocs.io/zh_CN/latest/_images/python-condition.png)

In [16]:
"""
Three threads print A B C in order.
"""
condition = Condition()
current = "A"

class ThreadA(Thread):
    def run(self):
        global current
        for _ in range(10):
            with condition:
                while current != "A":
                    condition.wait()
                print("A")
                current = "B"
                condition.notify_all()

class ThreadB(Thread):
    def run(self):
        global current
        for _ in range(10):
            with condition:
                while current != "B":
                    condition.wait()
                print("B")
                current = "C"
                condition.notify_all()

class ThreadC(Thread):
    def run(self):
        global current
        for _ in range(10):
            with condition:
                while current != "C":
                    condition.wait()
                print("C")
                current = "A"
                condition.notify_all()

a = ThreadA()
b = ThreadB()
c = ThreadC()

a.start()
b.start()
c.start()

a.join()
b.join()
c.join()

A
B
C
A
B
C
A
B
C
A
B
C
A
B
C
A
B
C
A
B
C
A
B
C
A
B
C
A
B
C


## 使用事件进行线程同步
- 事件是线程之间用于通讯的对象。有的线程等待信号，有的线程发出信号。基本上事件对象都会维护一个内部变量，可以通过 set() 方法设置为 true ，也可以通过 clear() 方法设置为 false 。 wait() 方法将会阻塞线程，直到内部变量为 true 
- ![pic](https://python-parallel-programmning-cookbook.readthedocs.io/zh_CN/latest/_images/event.png)

In [2]:
items = []
event = Event()

class Consumer(Thread):
    def __init__(self, items, event):
        super().__init__()
        self.items = items
        self.event = event
        
    def run(self):
        while True:
            time.sleep(2)
            self.event.wait()
            item = self.items.pop()
            print(f'Consumer notify: {item} popped from list by {self.name}')
            
class Producer(Thread):
    def __init__(self, items, event):
        super().__init__()
        self.items = items
        self.event = event
        
    def run(self):
        global item
        for _ in range(10):
            time.sleep(2)
            item = random.randint(0,256)
            self.items.append(item)
            print(f'Producer notify: item N {item} append to list by {self.name}')
            print(f'Producer notify: event set by {self.name}')
            self.event.set()
            print(f'Producer notify: event cleared by {self.name}')
            self.event.clear() 

In [ ]:
t1 = Producer(items, event)
t2 = Consumer(items, event)
t1.start()
t2.start()
t1.join()
t2.join()
print(f'{current_thread().name} exited...')

Producer notify: item N 77 append to list by Thread-6
Producer notify: event set by Thread-6
Producer notify: event cleared by Thread-6Consumer notify: 77 popped from list by Thread-7

Producer notify: item N 173 append to list by Thread-6
Producer notify: event set by Thread-6
Producer notify: event cleared by Thread-6
Consumer notify: 173 popped from list by Thread-7
Producer notify: item N 75 append to list by Thread-6
Producer notify: event set by Thread-6
Producer notify: event cleared by Thread-6
Consumer notify: 75 popped from list by Thread-7
Producer notify: item N 54 append to list by Thread-6
Producer notify: event set by Thread-6
Producer notify: event cleared by Thread-6
Consumer notify: 54 popped from list by Thread-7
Producer notify: item N 70 append to list by Thread-6
Producer notify: event set by Thread-6
Producer notify: event cleared by Thread-6
Consumer notify: 70 popped from list by Thread-7
Producer notify: item N 205 append to list by Thread-6
Producer notify: e

# 使用`with`语法

In [3]:
logging.basicConfig(level=logging.DEBUG, format='(%(threadName)-10s) %(message)s',)

In [4]:
def threading_with(statement):
    with statement:
        logging.debug(f"{statement} acquired via with")
        
def threading_not_with(statement):
    statement.acquire()
    try:
        logging.debug(f"{statement} acquired directly")
    finally:
        statement.release()

In [5]:
lock = Lock()
rlock = RLock()
condition = Condition()
mutex = Semaphore(1)
syn_list = [lock, rlock, condition, mutex]

In [6]:
for statement in syn_list:
    t1 = Thread(target=threading_with, args=(statement, ))
    t2 = Thread(target=threading_not_with, args=(statement,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()

(Thread-6  ) <locked _thread.lock object at 0x0000021897BC4A30> acquired via with
(Thread-7  ) <locked _thread.lock object at 0x0000021897BC4A30> acquired directly
(Thread-8  ) <locked _thread.RLock object owner=21412 count=1 at 0x0000021897BC4878> acquired via with
(Thread-9  ) <locked _thread.RLock object owner=1368 count=1 at 0x0000021897BC4878> acquired directly
(Thread-10 ) <Condition(<locked _thread.RLock object owner=15308 count=1 at 0x0000021896DD01E8>, 0)> acquired via with
(Thread-11 ) <Condition(<locked _thread.RLock object owner=7224 count=1 at 0x0000021896DD01E8>, 0)> acquired directly
(Thread-12 ) <threading.Semaphore object at 0x00000218971C97F0> acquired via with
(Thread-13 ) <threading.Semaphore object at 0x00000218971C97F0> acquired directly


# 使用`queue`进行线程通信
- 前面我们已经讨论到，当线程之间如果要共享资源或数据的时候，可能变的非常复杂。如你所见，Python的threading模块提供了很多同步原语，包括信号量，条件变量，事件和锁。如果可以使用这些原语的话，应该优先考虑使用这些，而不是使用queue（队列）模块。队列操作起来更容易，也使多线程编程更安全，因为队列可以将资源的使用通过单线程进行完全控制，并且允许使用更加整洁和可读性更高的设计模式。

- Queue常用的方法有以下四个：
    - put(): 往queue中放一个item
    - get(): 从queue删除一个item，并返回删除的这个item
    - task_done(): 每次item被处理的时候需要调用这个方法
    - join(): 所有item都被处理之前一直阻塞
- ![pic](https://python-parallel-programmning-cookbook.readthedocs.io/zh_CN/latest/_images/queue.png)

In [2]:
from queue import Queue

In [18]:
class Producer(Thread):
    def __init__(self, queue):
        super().__init__()
        self.queue = queue
        
    def run(self):
        for _ in range(5):
            item = random.randint(0, 256)
            self.queue.put(item)
            print(f'Producer notify: item N {item} appended to queue by {self.name}\n')
            time.sleep(1)
            
class Consumer(Thread):
    def __init__(self, queue):
        super().__init__()
        self.queue = queue
        
    def run(self):
        while True:
            try:
                # wait for 10s if queue is still empty then break the loop
                item = self.queue.get(timeout=10)   
                print(f"Consumer notify: {item} popped from queue by {self.name}")
                self.queue.task_done()
            except:
                break

In [19]:
queue = Queue()
t1 = Producer(queue)
t2 = Consumer(queue)
t3 = Consumer(queue)
t4 = Consumer(queue)
t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print(f"{current_thread().name} exited..")

Producer notify: item N 42 appended to queue by Thread-34

Consumer notify: 42 popped from queue by Thread-35
Producer notify: item N 37 appended to queue by Thread-34
Consumer notify: 37 popped from queue by Thread-35

Producer notify: item N 136 appended to queue by Thread-34
Consumer notify: 136 popped from queue by Thread-36

Producer notify: item N 179 appended to queue by Thread-34
Consumer notify: 179 popped from queue by Thread-36

Producer notify: item N 86 appended to queue by Thread-34
Consumer notify: 86 popped from queue by Thread-36

MainThread exited..


In [20]:
queue.queue

deque([])

# 评估多线程的性能
- GIL是CPython解释器引入的锁，GIL在解释器层面阻止了真正的并行运行。解释器在执行任何线程之前，必须等待当前正在运行的线程释放GIL。事实上，解释器会强迫想要运行的线程必须拿到GIL才能访问解释器的任何资源，例如栈或Python对象等。这也正是GIL的目的——阻止不同的线程并发访问Python对象。这样GIL可以保护解释器的内存，让垃圾回收工作正常。但事实上，这却造成了程序员无法通过并行执行多线程来提高程序的性能。如果我们去掉CPython的GIL，就可以让多线程真正并行执行。GIL并没有影响多处理器并行的线程，只是限制了一个解释器只能有一个线程在运行。

In [2]:
import sys
from timeit import Timer

In [3]:
class threads_object(Thread):
    def run(self):
        function_to_run()
        
class nothreads_object:
    def run(self):
        function_to_run()
        
def non_threaded(num_iter):
    funcs = []
    for _ in range(int(num_iter)):
        funcs.append(nothreads_object())
    for i in funcs:
        i.run()
        
def threaded(num_threads):
    funcs = []
    for _ in range(int(num_threads)):
        funcs.append(threads_object())
    for i in funcs:
        i.start()
    for i in funcs:
        i.join()

def show_results(func_name, results):
#     print(f"{func_name:-23s} {results :4.6f} seconds")
    print("%-23s %4.6f seconds" % (func_name, results))

## 第一次测试 - 空函数
- 多线程无收益

In [28]:
def function_to_run():
    pass

repeat = 100
number = 1
num_threads = [1, 2, 4, 8]
print('Start testing')
for i in num_threads:
    t = Timer(f"non_threaded({i})", 'from __main__ import non_threaded')
    best_result = min(t.repeat(repeat=repeat, number=number))
    show_results(f"non_threaded ({i} iters)", best_result)
    
    t = Timer("threaded(%s)" % i, "from __main__ import threaded")
    best_result = min(t.repeat(repeat=repeat, number=number))
    show_results("threaded (%s threads)" % i, best_result)
    print('Iterations complete')

Start testing
non_threaded (1 iters)  0.000001 seconds
threaded (1 threads)    0.000167 seconds
Iterations complete
non_threaded (2 iters)  0.000002 seconds
threaded (2 threads)    0.000316 seconds
Iterations complete
non_threaded (4 iters)  0.000002 seconds
threaded (4 threads)    0.000608 seconds
Iterations complete
non_threaded (8 iters)  0.000004 seconds
threaded (8 threads)    0.001227 seconds
Iterations complete


## 第二次测试 - 斐波那契数列
- 多线程没有收益：GIL和线程管理代码的开销
    - GIL只允许解释器一次执行一个线程

In [29]:
def function_to_run():
    a, b = 0, 1
    for i in range(10000):
        a, b = b, a + b

repeat = 100
number = 1
num_threads = [1, 2, 4, 8]
print('Start testing')
for i in num_threads:
    t = Timer(f"non_threaded({i})", 'from __main__ import non_threaded')
    best_result = min(t.repeat(repeat=repeat, number=number))
    show_results(f"non_threaded ({i} iters)", best_result)
    
    t = Timer("threaded(%s)" % i, "from __main__ import threaded")
    best_result = min(t.repeat(repeat=repeat, number=number))
    show_results("threaded (%s threads)" % i, best_result)
    print('Iterations complete')

Start testing
non_threaded (1 iters)  0.001978 seconds
threaded (1 threads)    0.002302 seconds
Iterations complete
non_threaded (2 iters)  0.003962 seconds
threaded (2 threads)    0.004645 seconds
Iterations complete
non_threaded (4 iters)  0.008026 seconds
threaded (4 threads)    0.009447 seconds
Iterations complete
non_threaded (8 iters)  0.016377 seconds
threaded (8 threads)    0.018876 seconds
Iterations complete


## 第三次测试 - 本地数据读取
- 测试读1kb的数据1000次

In [30]:
def function_to_run():
    fh=open("data/test.txt","r")
    size = 1024
    for i in range(1000):
        fh.read(size)
        
repeat = 100
number = 1
num_threads = [1, 2, 4, 8]
print('Start testing')
for i in num_threads:
    t = Timer(f"non_threaded({i})", 'from __main__ import non_threaded')
    best_result = min(t.repeat(repeat=repeat, number=number))
    show_results(f"non_threaded ({i} iters)", best_result)
    
    t = Timer("threaded(%s)" % i, "from __main__ import threaded")
    best_result = min(t.repeat(repeat=repeat, number=number))
    show_results("threaded (%s threads)" % i, best_result)
    print('Iterations complete')

Start testing
non_threaded (1 iters)  0.005919 seconds
threaded (1 threads)    0.006442 seconds
Iterations complete
non_threaded (2 iters)  0.012046 seconds
threaded (2 threads)    0.017330 seconds
Iterations complete
non_threaded (4 iters)  0.024811 seconds
threaded (4 threads)    0.042586 seconds
Iterations complete
non_threaded (8 iters)  0.049676 seconds
threaded (8 threads)    0.086268 seconds
Iterations complete


## 第四次测试 - 网络数据读取
- 从网页上获取数据并读取1k数据

In [ ]:
def function_to_run():
    import requests
    url='https://python-parallel-programmning-cookbook.readthedocs.io/zh_CN/latest/chapter2/13_Evaluating_the_performance_of_multithread_applications.html'
    for _ in range(10):
        r = requests.get(url)
        r.encoding = 'utf-8'
        r.text[:1024]
        
repeat = 100
number = 1
num_threads = [1, 2, 4, 8]
print('Start testing')
for i in num_threads:
    t = Timer(f"non_threaded({i})", 'from __main__ import non_threaded')
    best_result = min(t.repeat(repeat=repeat, number=number))
    show_results(f"non_threaded ({i} iters)", best_result)
    
    t = Timer("threaded(%s)" % i, "from __main__ import threaded")
    best_result = min(t.repeat(repeat=repeat, number=number))
    show_results("threaded (%s threads)" % i, best_result)
    print('Iterations complete')

Start testing


In [1]:
16 *13

208